In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing import image
from sklearn.model_selection import train_test_split
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [3]:
import tensorflow as tf
print(tf.__version__)

2.17.0


In [4]:
# Cek file CSV dan gambar (opsional, jika ada CSV)
csv_file_path = 'data/cats.csv'  # Path ke file CSV (jika ada)
images_folder_path = 'images/'    # Path ke folder images

# Cek apakah file CSV dan folder images ada
if os.path.exists(csv_file_path) and os.path.exists(images_folder_path):
    print("File CSV dan folder gambar tersedia.")
else:
    print("Pastikan file CSV dan folder gambar berada pada path yang benar.")

# Membaca file CSV (jika perlu)
cats_df = pd.read_csv(csv_file_path)
cats_df.head()


File CSV dan folder gambar tersedia.


,Unnamed: 0,id,url,type,age,gender,size,coat,breed,photos,med_photos
0,0,46744842,https://www.petfinder.com/cat/arianna-46744842...,Cat,Adult,Female,Medium,NaN,Abyssinian,[{'small': 'https://dl5zpyw5k3jeb.cloudfront.n...,['https://dl5zpyw5k3jeb.cloudfront.net/photos/...
1,1,46717321,https://www.petfinder.com/cat/ozzy-46717321/oh...,Cat,Adult,Male,Extra Large,NaN,Abyssinian,[{'small': 'https://dl5zpyw5k3jeb.cloudfront.n...,['https://dl5zpyw5k3jeb.cloudfront.net/photos/...
2,2,46626338,https://www.petfinder.com/cat/zena-46626338/ca...,Cat,Senior,Female,Medium,NaN,Abyssinian,[{'small': 'https://dl5zpyw5k3jeb.cloudfront.n...,['https://dl5zpyw5k3jeb.cloudfront.net/photos/...
3,3,46620170,https://www.petfinder.com/cat/mika-46620170/ca...,Cat,Baby,Female,Small,Short,Abyssinian,[{'small': 'https://dl5zpyw5k3jeb.cloudfront.n...,['https://dl5zpyw5k3jeb.cloudfront.net/photos/...
4,4,46582751,https://www.petfinder.com/cat/ruby-46582751/fl...,Cat,Young,Female,Medium,NaN,Abyssinian,[{'small': 'https://dl5zpyw5k3jeb.cloudfront.n...,['https://dl5zpyw5k3jeb.cloudfront.net/photos/...


In [5]:
# Preprocessing dan augmentasi gambar dengan ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./255,         # Normalisasi nilai pixel antara 0 dan 1
    shear_range=0.2,        # Shear augmentation
    zoom_range=0.2,         # Zoom augmentation
    horizontal_flip=True,   # Flip gambar secara horizontal
    validation_split=0.2    # Memisahkan 20% data untuk validasi
)

# Ukuran gambar yang akan digunakan dalam model CNN
image_size = (64, 64)

# Membuat generator dataset dari folder images (training dan validation set)
train_set = train_datagen.flow_from_directory(
    images_folder_path,
    target_size=image_size,
    batch_size=32,
    class_mode='categorical',
    subset='training'  # Subset training
)

validation_set = train_datagen.flow_from_directory(
    images_folder_path,
    target_size=image_size,
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # Subset validasi
)

# Print jumlah class (breed kucing)
print(f"Number of classes (cat breeds): {len(train_set.class_indices)}")

Found 101313 images belonging to 67 classes.
Found 25294 images belonging to 67 classes.
Number of classes (cat breeds): 67


In [6]:
# Membuat arsitektur CNN untuk klasifikasi breed kucing
model = Sequential()

# Layer 1 - Convolution + MaxPooling
model.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Layer 2 - Convolution + MaxPooling
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Layer 3 - Convolution + MaxPooling
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Layer Flatten
model.add(Flatten())

# Fully Connected Layer (Dense)
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  # Menggunakan Dropout untuk regularisasi

# Output Layer (Softmax untuk multiclass classification)
model.add(Dense(len(train_set.class_indices), activation='softmax'))

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Melihat ringkasan arsitektur model
model.summary()


c:\python\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       589,952 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 67)             │         8,643 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 691,843 (2.64 MB)

 Trainable params: 691,843 (2.64 MB)

 Non-trainable params: 0 (0.00 B)

In [40]:
# Melatih model tanpa menyetel steps_per_epoch
history = model.fit(
    train_set,
    epochs=25,  # Ubah jumlah epochs sesuai kebutuhan
    validation_data=validation_set
)


Epoch 1/25


c:\python\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


3167/3167 ━━━━━━━━━━━━━━━━━━━━ 517s 162ms/step - accuracy: 0.4159 - loss: 2.7415 - val_accuracy: 0.4193 - val_loss: 2.4999
Epoch 2/25
3167/3167 ━━━━━━━━━━━━━━━━━━━━ 1130s 356ms/step - accuracy: 0.4190 - loss: 2.5454 - val_accuracy: 0.4193 - val_loss: 2.4398
Epoch 3/25
3167/3167 ━━━━━━━━━━━━━━━━━━━━ 1077s 340ms/step - accuracy: 0.4166 - loss: 2.4879 - val_accuracy: 0.4214 - val_loss: 2.3837
Epoch 4/25
3167/3167 ━━━━━━━━━━━━━━━━━━━━ 1234s 389ms/step - accuracy: 0.4196 - loss: 2.4348 - val_accuracy: 0.4203 - val_loss: 2.3634
Epoch 5/25
3167/3167 ━━━━━━━━━━━━━━━━━━━━ 1159s 365ms/step - accuracy: 0.4217 - loss: 2.4015 - val_accuracy: 0.4237 - val_loss: 2.3797
Epoch 6/25
3167/3167 ━━━━━━━━━━━━━━━━━━━━ 31258s 10s/step - accuracy: 0.4237 - loss: 2.3751 - val_accuracy: 0.4276 - val_loss: 2.3111
Epoch 7/25
3167/3167 ━━━━━━━━━━━━━━━━━━━━ 483s 152ms/step - accuracy: 0.4237 - loss: 2.3573 - val_accuracy: 0.4286 - val_loss: 2.3211
Epoch 8/25
3167/3167 ━━━━━━━━━━━━━━━━━━━━ 470s 148ms/step - accuracy:

In [7]:
# Evaluasi model menggunakan validation set
val_loss, val_acc = model.evaluate(validation_set)
print(f"Validation Accuracy: {val_acc * 100:.2f}%")


c:\python\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


791/791 ━━━━━━━━━━━━━━━━━━━━ 83s 104ms/step - accuracy: 0.0163 - loss: 4.1751
Validation Accuracy: 1.55%


In [9]:
# Menyimpan model setelah training selesai
model.save('cat_breed_classifier.h5')
print("Model saved successfully!")


Model saved successfully!


In [21]:
# Fungsi untuk memprediksi breed kucing dari gambar baru
def predict_breed(img_path):
    img = image.load_img(img_path, target_size=(64, 64))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0

    # Melakukan prediksi
    prediction = model.predict(img_array)
    breed_index = np.argmax(prediction)
    
    # Mapping indeks ke nama breed
    class_labels = list(train_set.class_indices.keys())
    predicted_breed = class_labels[breed_index]

    return predicted_breed

# Contoh penggunaan fungsi prediksi
image_path = 'images/Sphynx - Hairless Cat/18778434_293.jpg'  # Path ke gambar uji
predicted_breed = predict_breed(image_path)
print(f"Predicted Breed: {predicted_breed}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Predicted Breed: American Shorthair
